<a href="https://colab.research.google.com/github/piyush7911/sentiment-model-/blob/main/sentiment_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


In [ ]:
!pip install scikit-learn

In [ ]:
import pandas as pd
pd.set_option("display.max_colwidth", 200)
data = pd.read_csv('/content/sampledata.csv')

data.head()


In [4]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import nltk
nltk.download('punkt')
nltk.download('stopwords')
!pip install contractions
import contractions
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 10.4 MB/s eta 0:00:00


[nltk_data] Downloading package wordnet to /root/nltk_data...


In [5]:
from nltk.corpus import stopwords
nltk_stopwords = set(stopwords.words('english'))
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
sklearn_stopwords = set(ENGLISH_STOP_WORDS)
combined_stopwords = nltk_stopwords.union(sklearn_stopwords)

In [6]:
import re
def tweet_cleaner_with_stopwords(text):
    new_text = re.sub("#", "",text)
    new_text = re.sub("@[A-Za-z0-9]+", "", new_text)
    new_text = re.sub(r"http\S+", "", new_text)
    new_text = contractions.fix(new_text)
    new_text = re.sub(r"[^a-zA-Z]", " ", new_text)
    new_text = new_text.lower().strip()

    new_text = [token for token in new_text.split() ]

    new_text = [token for token in new_text if len(token)>2]

    cleaned_text = ''
    for token in new_text:
        cleaned_text = cleaned_text + lemmatizer.lemmatize(token) + ' '

    return cleaned_text

In [7]:

cleaned_tweets = list(data['text'].apply(tweet_cleaner_with_stopwords))
data['cleaned_tweets_with_SW'] = cleaned_tweets

print(cleaned_tweets[:10])

['having relaxing day the beach serenity ', 'received unexpected compliment feeling appreciated boosted ', 'dealing with leaky roof frustrating ', 'excited about upcoming art exhibition artistry ', 'spent the day the amusement park excitement ', 'locked myself out the house not again frustration ', 'enjoying delicious meal with friend goodcompany ', 'had heartfelt conversation with loved one emotional ', 'attended family picnic joyful ', 'missed important call feeling regretful ']


In [8]:
data.head()

,text,sentiment,cleaned_tweets_with_SW
0,Having a relaxing day at the beach. #Serenity,Positive,having relaxing day the beach serenity
1,Received an unexpected compliment. Feeling appreciated! #Boosted,Positive,received unexpected compliment feeling appreciated boosted
2,Dealing with a leaky roof. Frustrating!,Negative,dealing with leaky roof frustrating
3,Excited about my upcoming art exhibition. #Artistry,Positive,excited about upcoming art exhibition artistry
4,Spent the day at the amusement park. #Excitement,Positive,spent the day the amusement park excitement


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=5000)

tfidf_matrix = tfidf_vectorizer.fit_transform(cleaned_tweets)

tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

data_with_tfidf = pd.concat([data, tfidf_df], axis=1)

data_with_tfidf.head()

,text,sentiment,cleaned_tweets_with_SW,about,accomplished,aced,admirer,adopted,adventure,adventureawaits,...,will,with,won,work,workout,workshop,worst,would,wrong,zoo
0,Having a relaxing day at the beach. #Serenity,Positive,having relaxing day the beach serenity,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Received an unexpected compliment. Feeling appreciated! #Boosted,Positive,received unexpected compliment feeling appreciated boosted,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Dealing with a leaky roof. Frustrating!,Negative,dealing with leaky roof frustrating,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.30893,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Excited about my upcoming art exhibition. #Artistry,Positive,excited about upcoming art exhibition artistry,0.339492,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spent the day at the amusement park. #Excitement,Positive,spent the day the amusement park excitement,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(data['sentiment'])

X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, y_encoded, test_size=0.2, random_state=42)

classifier = MultinomialNB()

classifier.fit(X_train, y_train)



MultinomialNB()

In [11]:

y_pred = classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

report = classification_report(y_test, y_pred, target_names=["0", "1"])
print(report)


Accuracy: 0.9888888888888889
Precision: 0.9891891891891892
Recall: 0.9888888888888889
F1-Score: 0.9889130713096915
              precision    recall  f1-score   support

           0       0.97      1.00      0.99        36
           1       1.00      0.98      0.99        54

    accuracy                           0.99        90
   macro avg       0.99      0.99      0.99        90
weighted avg       0.99      0.99      0.99        90



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pickle
model_data = {'model': classifier, 'vectorizer': tfidf_vectorizer}
with open('/content/drive/My Drive/sentiment_model.pkl', 'wb') as f:
    pickle.dump(model_data, f)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

sample_text = "today is a joyfull day"

cleaned_sample_text = tweet_cleaner_with_stopwords(sample_text)

sample_tfidf = tfidf_vectorizer.transform([cleaned_sample_text])

sample_prediction = classifier.predict(sample_tfidf)

sample_prediction_label = label_encoder.inverse_transform(sample_prediction)

print("Sample Text:", sample_text ,' , ','Cleanend Text:', cleaned_sample_text)
print("Predicted Sentiment:", sample_prediction_label)


Sample Text: today is a joyfull day  ,  Cleanend Text: today joyfull day 
Predicted Sentiment: [' Positive']
